In [ ]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [ ]:
from pathlib import Path
import numpy
#import np_utils

# Reads the files and folders within the MIDI folder
# and stores the names in the list
songs = []
folder = Path('/content/drive/MyDrive/MIDI/MIDI/')
for file in folder.rglob('*.mid'):
  songs.append(file)

In [ ]:
print(f'Number of songs in the dataset: {len(songs)}')

17230

In [ ]:
# Get a sample of the vast list of songs

import random
result =  random.sample([x for x in songs], 5000)

In [ ]:
# Convert the files from .mid to a list of all the notes.

from music21 import converter, instrument, note, chord
notes = []
for i,file in enumerate(result):
    print(f'{i+1}: {file}')
    try:
      midi = converter.parse(file)
      notes_to_parse = None
      parts = instrument.partitionByInstrument(midi)
      if parts: # file has instrument parts
          notes_to_parse = parts.parts[0].recurse()
      else: # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))
    except:
      print(f'FAILED: {i+1}: {file}')

1: /content/drive/MyDrive/MIDI/MIDI/Paoli/Quattro_amici.mid
2: /content/drive/MyDrive/MIDI/MIDI/Collins_Phil/I_Dont_Care_Anymore.mid
3: /content/drive/MyDrive/MIDI/MIDI/The_Cranberries/Dreams.1.mid
4: /content/drive/MyDrive/MIDI/MIDI/Carole_King/So_Far_Away.mid
5: /content/drive/MyDrive/MIDI/MIDI/Petula_Clark/Downtown.1.mid
6: /content/drive/MyDrive/MIDI/MIDI/a-ha/Take_On_Me.2.mid
FAILED: 6: /content/drive/MyDrive/MIDI/MIDI/a-ha/Take_On_Me.2.mid
7: /content/drive/MyDrive/MIDI/MIDI/Tom_Jones/Green_Green_Grass_of_Home.2.mid
8: /content/drive/MyDrive/MIDI/MIDI/Van_Morrison/Have_I_Told_You_Lately.2.mid
9: /content/drive/MyDrive/MIDI/MIDI/Rossi_Vasco/Liberi..._liberi.1.mid
FAILED: 9: /content/drive/MyDrive/MIDI/MIDI/Rossi_Vasco/Liberi..._liberi.1.mid
10: /content/drive/MyDrive/MIDI/MIDI/REDDING_OTIS/Sitin_on_The_Dock_of_the_Bay.mid
FAILED: 10: /content/drive/MyDrive/MIDI/MIDI/REDDING_OTIS/Sitin_on_The_Dock_of_the_Bay.mid
11: /content/drive/MyDrive/MIDI/MIDI/Pooh/Ci_pensero_domani.mid
FAILED

In [ ]:
# Save notes to Drive for future usage
with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

In [ ]:
# Load notes from Drive
with open('/content/drive/MyDrive/notes', 'rb') as filepath:
  notes = pickle.load(filepath)

In [ ]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all unique pitchnames
    pitchnames = sorted(set(item for item in notes))

     # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        # sequence_in is a sequence_length list containing sequence_length notes
        sequence_in = notes[i:i + sequence_length]
        # sequence_out is the sequence_length + 1 note that comes after all the notes in
        # sequence_in. This is so the model can read sequence_length notes before predicting
        # the next one.
        sequence_out = notes[i + sequence_length]
        # network_input is the same as sequence_in but it containes the indexes from the notes
        # because the model is only fed the indexes.
        network_input.append([note_to_int[char] for char in sequence_in])
        # network_output containes the index of the sequence_out
        network_output.append(note_to_int[sequence_out])

    # n_patters is the length of the times it was iterated 
    # for example if i = 3, then n_patterns = 3
    # because network_input is a list of lists
    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    # Reshapes it into a n_patterns by sequence_length matrix
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    # OneHot encodes the network_output
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes,n_vocab)

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(Flatten())
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    #model.add(Dense(2048))
    model.add(Dense(1024))
    #model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

    return model
model = create_network(network_input, n_vocab)

In [ ]:
import tensorflow as tf

tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False
)

In [ ]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=5, batch_size=128, callbacks=callbacks_list)

    return model

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    # Selects a random row from the network_input
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # Random row from network_input
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        print(note_index)
        # Reshapes pattern into a vector
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        # Standarizes pattern
        prediction_input = prediction_input / float(n_vocab)

        # Predicts the next note
        prediction = model.predict(prediction_input, verbose=0)

        # Outputs a OneHot encoded vector, so this picks the columns
        # with the highest probability
        index = numpy.argmax(prediction)
        # Maps the note to its respective index
        result = int_to_note[index]
        # Appends the note to the prediction_output
        prediction_output.append(result)

        # Adds the predicted note to the pattern
        pattern = numpy.append(pattern,index)
        # Slices the array so that it contains the predicted note
        # eliminating the first from the array, so the model can
        # have a sequence
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
from music21 import instrument, note, stream

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')